In [1]:
import os

In [2]:
%pwd

'/home/tinashe/codes/chicken2/coccidiosis_identification/research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'/home/tinashe/codes/chicken2/coccidiosis_identification'

In [5]:
from dataclasses import dataclass
from pathlib import Path


#initialising  prepare_callbacks stuff
@dataclass(frozen=True)
class PrepareCallbacksConfig:
    root_dir: Path
    tensorboard_root_log_dir: Path
    checkpoint_model_filepath: Path


In [6]:
from classifier.constants import *
from classifier.utils.common_functions import read_yaml, create_dirctories

2024-12-09 04:57:12,516 - classifier - INFO - welcome tinashe


In [7]:
class ConfigurationManager:
    def __init__(
            self,
            config_filepath= CONFIG_FILE_PATH,
            parameters_filepath= PARAMETERS_FILE_PATH

    ):
        self.config=read_yaml(config_filepath)
        self.parameters= read_yaml(parameters_filepath)
        create_dirctories([self.config.artifacts_roots])
                

    def get_prepare_callbacks_config(self)-> PrepareCallbacksConfig:
        config=self.config.prepare_callbacks
        model_ckpt_dir= os.path.dirname(config.checkpoint_model_filepath)

        create_dirctories([
            Path(model_ckpt_dir),
            Path(config.tensorboard_root_log_dir)
        ])

        prepare_callbacks_config= PrepareCallbacksConfig(
            root_dir= Path(config.root_dir),
            tensorboard_root_log_dir=Path(config.tensorboard_root_log_dir),
            checkpoint_model_filepath=Path(config.checkpoint_model_filepath)
        )
        return prepare_callbacks_config

In [8]:
import os
import urllib.request as requests
from zipfile import ZipFile
import tensorflow as tf
import time

2024-12-09 04:57:13.793935: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [9]:
class PrepareCallbacks:
    def __init__(self, config: PrepareCallbacksConfig):
        self.config=config

    @property
    def _create_tb_callbacks(self):
        time_stamp= time.strftime("%Y-%M-%d-%H-%M-%S")
        tb_running_log_dir= os.path.join(
            self.config.tensorboard_root_log_dir,
            f"tb_logs_at_{time_stamp}",

        )
        return tf.keras.callbacks.TensorBoard(log_dir=tb_running_log_dir)

        


    @property
    def _create_ckpt_callbacks(self):
        return tf.keras.callbacks.ModelCheckpoint(
            filepath= self.config.checkpoint_model_filepath,
            save_best_only= True
        )
    


    
    def get_tb_ckpt_callbacks(self):
        return[
            self._create_tb_callbacks,
            self._create_ckpt_callbacks,
        ]


In [10]:

try:
    config=ConfigurationManager()
    prepare_callbacks_config= config.get_prepare_callbacks_config()
    prepare_callbacks= PrepareCallbacks(config=prepare_callbacks_config)
    callback_list= prepare_callbacks.get_tb_ckpt_callbacks()

except Exception as e:
    raise e

2024-12-09 04:57:18,270 - classifier - INFO - yaml file: config/config.yaml loaded succefully
2024-12-09 04:57:18,280 - classifier - INFO - yaml file: parameters.yaml loaded succefully
2024-12-09 04:57:18,292 - classifier - INFO - created a directory at: artifacts
2024-12-09 04:57:18,304 - classifier - INFO - created a directory at: artifacts/prepare_callbacks/checkpoint_dir
2024-12-09 04:57:18,310 - classifier - INFO - created a directory at: artifacts/prepare_callbacks/tensorboard_log_dir


In [11]:
import time
timestamp= time.strftime("%Y-%M-%d-%H-%M-%S")
f"tb_logs_at_{timestamp}"

'tb_logs_at_2024-57-09-04-57-18'